<a href="https://colab.research.google.com/github/AgustinRamiroDiaz/Algorithmic-Python-Trading/blob/main/RSI_Strat_x_500_Algovibes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install yfinance
!pip install matplotlib
!pip install ta

  Created wheel for ta: filename=ta-0.7.0-py3-none-any.whl size=28718 sha256=3e3a580869961ab54e9bee37b8b4a6a08c73e2ce932d8baca82561929c40e774
  Stored in directory: /home/az/.cache/pip/wheels/bb/7c/a0/9c72e50ddef1f7c3d9003bf4ccc5d5c8deb24828d4eb156fc8
Successfully built ta


In [37]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import ta
pd.options.mode.chained_assignment = None

In [ ]:
# Busco la lista de 500 tickers con sus datos en el site de wikipedia
allTablesInWikipedia = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
tickersWithExtraData = allTablesInWikipedia[0]

In [ ]:
# Imprimo la tabla de Wikipedia
tickersWithExtraData

In [ ]:
# Solo me interesa la columna 'Symbol'
tickers = tickersWithExtraData['Symbol'].to_list()

In [ ]:
tickers

In [ ]:
# los tickers que tienen un punto dan error, entonces los corrijo cambiando el punto por un guion
tickers = [x.replace('.','-') for x in tickers]

In [ ]:
tickers

In [ ]:
# Si quiero eliminar un ticker de la lista
tickers.remove('WDC')


# DESCARGA DE DATOS CON YF. DESCOMENTAR DE SER NECESARIO


In [ ]:
# fechaDesde = '2018-01-01'
# sp500 = yf.download(tickers, start = fechaDesde)       # No defino 'end' porque llega hasta hoy


In [ ]:
# sp500.to_pickle("./SP500-data.pkl")

In [ ]:
sp500 = pd.read_pickle("./SP500-data.pkl")

In [ ]:
sp500

In [ ]:
sp500.xs('MMM', axis=1, level=1)

In [49]:
# Defino una funcion para calcular el indicador.
# Es una funcion que hace todos los calculos y nos devuelve un dataframe con una columna con el resultado para cada linea

def RSIcalc(df):
    df['MA200'] = df['Adj Close'].rolling(window=200).mean()  
    
    df['RSI'] = ta.momentum.rsi(df["Adj Close"], window=14)

    # vamos a definir cuando generar la señal usando el comando df.loc (ubicar)
    # cuando se cumplan las condiciones definidas, genera una columna 'Buy' y le asigna el valor True o False
    # El shift es para que el Buy de True al día siguiente, que es cuando se tiene la información para poder realizar la compra
    df['Buy'] = (df['Adj Close'].shift(1) > df['MA200'].shift(1)) & (df['RSI'].shift(1) < 30) 
    
    return df

In [56]:
#@title Insertar ticker para ejecutar la función RSIcalc
tickerToProcess = "TSLA" #@param {type:"string"}
dfAsociated = df.xs(tickerToProcess, axis=1, level=1)
dfOutput = RSIcalc(dfAsociated)
dfOutput[dfOutput["Buy"] == True]

,Adj Close,Close,High,Low,Open,Volume,MA200,RSI,Buy
Date,,,,,,,,,
2020-03-17,86.040001,86.040001,94.370003,79.199997,88.001999,119973000.0,73.917960,27.653172,True
2020-03-18,72.244003,72.244003,80.972000,70.101997,77.800003,118931000.0,74.100210,24.282496,True
2021-03-08,563.000000,563.000000,620.130005,558.789978,600.549988,51787000.0,476.684739,26.800441,True
2021-03-09,673.580017,673.580017,678.090027,595.210022,608.179993,67523300.0,479.225039,44.538192,True


# Creamos una nueva funcion  que usa un dataframe como argumento y devuelve las fechas de compra y venta.
# Con esas fechas vamos a poder calcular P&L

In [107]:
def getSignals(df):
    buyDates = []
    sellDates = []

    # Hacemos loop con i para barrer todas las filas del dataframe
    for i in range(len(df) - 11 ):    # resta 11 xq una condicion de venta es 10 dias despues de la compra
    
        # si hay un 'Yes' en la columna 'Buy', agregamos al dataframe el nombre de la fila (que es la fecha)
        if df['Buy'].iloc[i]:          # ubica la fila con el 'Yes'
            buyDates.append(df.iloc[i].name)
            # para buscar la fecha de salida, la condicion es que RSI llegue a un determinado valor antes de los 10 dias
            # entonces usa un nuevo loop j de 1 a 11 (para q incluya hasta el 10)

            for j in range(0, 10):
                if j == 9 or df['RSI'].iloc[i+j] > 40:    # controla desde cada fila i y durante j filas si se cumple la condicion )o( si no se cumple la condicion de venta antes de los 10 dias, vende el dia 10
                    sellDates.append(df.iloc[i+j+1].name)
   # si se cumple, agrega la fecha de venta. El +1 idem anterior
                    break    # corta el loop cuando se cumple la condicion
                # si no se cumple la condicion de venta antes de los 10 dias, vende el dia 10
  

    return buyDates, sellDates    # devuelve el df original agregando las fechas de venta



In [108]:
buyDates, sellDates = getSignals(dfOutput)
signals = pd.DataFrame()
signals["Buy Date"] = buyDates
signals["Sell Date"] = sellDates
signals

,Buy Date,Sell Date
0,2020-03-17,2020-03-25
1,2020-03-18,2020-03-25
2,2021-03-08,2021-03-10
3,2021-03-09,2021-03-10


In [113]:
signalsFull = signals.join(dfOutput, on='Buy Date').join(dfOutput, on='Sell Date', rsuffix = ' Sell')
signalsFull.columns

Index(['Buy Date', 'Sell Date', 'Adj Close', 'Close', 'High', 'Low', 'Open',
       'Volume', 'MA200', 'RSI', 'Buy', 'Adj Close Sell', 'Close Sell',
       'High Sell', 'Low Sell', 'Open Sell', 'Volume Sell', 'MA200 Sell',
       'RSI Sell', 'Buy Sell'],
      dtype='object')

In [121]:
signalsView = signalsFull[['Buy Date', 'Open',
       'MA200', 'RSI', 'Sell Date', 'Open Sell','MA200 Sell',
       'RSI Sell']]
signalsView.round(2)

,Buy Date,Open,MA200,RSI,Sell Date,Open Sell,MA200 Sell,RSI Sell
0,2020-03-17,88.00,73.92,27.65,2020-03-25,109.05,75.42,45.79
1,2020-03-18,77.80,74.10,24.28,2020-03-25,109.05,75.42,45.79
2,2021-03-08,600.55,476.68,26.80,2021-03-10,700.30,481.75,43.97
3,2021-03-09,608.18,479.23,44.54,2021-03-10,700.30,481.75,43.97


In [ ]:
sellDates = frame[frame["Buy"] == True]["Sell Date"]
buyDates = frame[frame["Buy"] == True].axes[0]


buyDates
sellDates

In [ ]:
frame.loc[sellDates]

In [ ]:
# mostramos las señales de compra
plt.figure(figsize=(12,5))# tamano del plot
plt.scatter(
    frame.loc[buyDates].index, 
    frame.loc[buyDates]['Adj Close'], 
    marker = '^', 
    c='r'
    )
plt.plot(frame['Adj Close'], alpha = 0.7)


In [ ]:
frame["Win"] = frame["Profit"] > 0
frame

In [ ]:
wins = frame[frame["Win"] == True]
wins

In [ ]:
len(wins) / len(frame[frame["Buy"] == True])

In [ ]:
len(frame[frame["Buy"] == True])   # cantidad de transacciones

In [ ]:
# Ahora pasemos a las 500 tickers

matrixsignals = []
matrixprofits = []

for i in range(len(tickers)):
  frame = RSIcalc(tickers[i])
  buy, sell = getSignals(frame)
  Profits = (frame.loc[sell].Open.values -frame.loc[buy].Open.values)/ frame.loc[buy].Open.values
  matrixsignals.append(buy)
  matrixprofits.append(Profits)


In [ ]:
matrixsignals

In [ ]:
matrixprofits

In [ ]:
len(matrixprofits)

In [ ]:
# Crea una lista 'allprofit'
allprofit = []

for i in matrixprofits:
  for e in i:
    allprofit.append(e)

In [ ]:
wins = [i for i in allprofit if i>0]

In [ ]:
len(wins)/len(allprofit)

In [ ]:
plt.hist(allprofit, bins=100)
plt.show

In [ ]:
for i in matrixsignals:
  for e in i:
    if e.year == 2021:
      print(e)